In [12]:
from osgeo import gdal, ogr
import geopandas as gpd


In [30]:
# Define pixel_size and NoData value of new raster
pixel_size = 30
NoData_value = -9999

# Filename of input OGR file
vector_fn = '/Users/yamaoka/landuse/labels/texas-latest-free.shp/gis.osm_buildings_a_free_1_austin.shp'

# Filename of the raster Tiff that will be created
raster_fn = '/Users/yamaoka/test_raster.tif'

# Open the data source and read in the extent
#tmp_ds = gpd.GeoDataFrame.from_file(vector_fn)
#new_ds = tmp_ds.to_crs({'init':'epsg:32614'})
#new_ds.to_file('tempepsg3214.shp')

source_ds = ogr.Open('tempepsg3214.shp')
source_layer = source_ds.GetLayer(0)
x_min, x_max, y_min, y_max = source_layer.GetExtent()
#print (x_min, x_max, y_min, y_max)
# Create the destination data source
x_res = int((x_max - x_min) / pixel_size)
y_res = int((y_max - y_min) / pixel_size)

driver = gdal.GetDriverByName('GTiff')
target_ds = driver.Create(raster_fn, x_res, y_res, 1, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_min, pixel_size, 0, y_max, 0, -pixel_size))
band = target_ds.GetRasterBand(1)
band.SetNoDataValue(NoData_value)

# Rasterize
gdal.RasterizeLayer(target_ds, [1], source_layer, burn_values=[0])

0

In [ ]:
import rasterio

with rasterio.open('rtest.tif', 'w', **meta) as out:
    out_arr = out.read(1)

    # this is where we create a generator of geom, value pairs to use in rasterizing
    #shapes = ((geom,value) for geom, value in zip(counties.geometry, counties.LSAD_NUM))

    burned = rasterio.features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)
    out.write_band(1, burned)